In [ ]:
import sys
!{sys.executable} -m pip install openai==1.1.1 tiktoken

In [64]:
import sys
!{sys.executable} -m pip show openai

Name: openai
Version: 1.1.1
Summary: Client library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages
Requires: anyio, distro, httpx, pydantic, tqdm, typing-extensions
Required-by: gpt-engineer, gpt-index, litellm, llama-index, open-interpreter


In [65]:
import os
import dotenv

# Get the path to the .env file
dotenv_path = dotenv.find_dotenv()

# Load the .env file
dotenv.load_dotenv(dotenv_path)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [66]:
import base64
import time
import openai
import os
import requests
import json

In [67]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

GPT-4 Turbo is our latest generation model. It’s more capable, has an updated knowledge cutoff of April 2023 and introduces a 128k context window (the equivalent of 300 pages of text in a single prompt). The model is also 3X cheaper for input tokens and 2X cheaper for output tokens compared to the original GPT-4 model. The maximum number of output tokens for this model is 4096.

In [68]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What are some of the key events that happened in March 2023?",
        }
    ],
    model="gpt-4-1106-preview",
)

InternalServerError: Error code: 502 - {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}}

In [ ]:
print(completion.choices[0].message.content)

In [ ]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What are some of the key events that happened Janurary 2023 for New Zealand Prime Minister Jacinda Ardern?",
        }
    ],
    model="gpt-3.5-turbo-1106",
)

In [ ]:
print(completion.choices[0].message.content)

JSON Mode

By setting response_format: { type: "json_object" } when calling gpt-4-visual-preview or gpt-3.5-turbo, these models will constrain their output to generate strings that parse into valid JSON. 

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={"type": "json_object"},
  messages=[
    {"role": "system", "content": "You are a helpful programmer who always returns your answer in JSON."},
    {"role": "user", "content": "give me a list of 5 things for grocery shopping. call the list 'groceries'"}
  ]
)

print(completion.choices[0].message)
print(completion)


In [ ]:
import json
# Use json.dumps() to pretty print the JSON
pretty_json = json.dumps(completion.choices[0].message, indent=4)

# Print the pretty JSON
print(pretty_json)

DALL-E 3

In [ ]:
PROMPT = "a room full of dogs all meditating in a circle"

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.images.generate(
  model="dall-e-3",
  prompt= PROMPT,
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
image_url

In [ ]:
import requests
from PIL import Image
from io import BytesIO
from IPython.display import display

def display_and_save_image_from_url(url, scale_percent=100, save_name='image.png'):
    # Send a GET request to the specified URL to retrieve the image
    response = requests.get(url)
    # Open the image
    img = Image.open(BytesIO(response.content))

    # Calculate the new size, as a percentage of the original size
    if scale_percent != 100:
        width, height = img.size
        new_width = int(width * scale_percent / 100)
        new_height = int(height * scale_percent / 100)
        img = img.resize((new_width, new_height), Image.ANTIALIAS)

    # Save the image locally with the given name
    img.save(save_name)

    # Display the image in the notebook
    display(img)

# Call the function with the URL, the scale percentage, and the save name you want
display_and_save_image_from_url(image_url, scale_percent=50, save_name='../data/meditating_dogs.png')

GPT Voice


In [61]:
import base64
import requests

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "../data/meditating_dogs.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}"
}

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Describe what’s in this image in detail as a story?"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-8IO9J4hIPPGV2mdRiXValAua2c1G4', 'object': 'chat.completion', 'created': 1699393425, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 274, 'completion_tokens': 300, 'total_tokens': 574}, 'choices': [{'message': {'role': 'assistant', 'content': 'As the sun begins its descent, painting the sky with amber hues, a warm and cozy room bathed in the gentle embrace of the evening light becomes the stage for a gathering most extraordinary. A circle of dogs, each with a coat as soft and plush as the finest velvet, sits in attentive silence. The glow from the setting sun streams through the window, casting a golden sheen over their diverse textures and colors.\n\nThese are not your ordinary canines. They are the epitome of grace and serenity, an unexpected assembly within what appears to be a tranquil home adorned with wooden panels and a homely vibe. A group of pillows lies in a circle, indicating this is a place of rest and reflection.\n\nIn the center of this re

In [62]:
story = response.json()['choices'][0]['message']['content']

print(story)

As the sun begins its descent, painting the sky with amber hues, a warm and cozy room bathed in the gentle embrace of the evening light becomes the stage for a gathering most extraordinary. A circle of dogs, each with a coat as soft and plush as the finest velvet, sits in attentive silence. The glow from the setting sun streams through the window, casting a golden sheen over their diverse textures and colors.

These are not your ordinary canines. They are the epitome of grace and serenity, an unexpected assembly within what appears to be a tranquil home adorned with wooden panels and a homely vibe. A group of pillows lies in a circle, indicating this is a place of rest and reflection.

In the center of this respectful congregation, one particular dog stands out, its back turned to us, confidently addressing its attentive audience. This central figure seems to command the room with an air of wisdom and calm authority, suggesting that this gathering might very well be a council of the ca

## Text To Speech


In [63]:
from IPython.display import Audio
from pathlib import Path

response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input=story
)

# Define the path where you want to save the file
speech_file_path = Path('../data/story.mp3')

# Save the response content (binary content of the mp3 file) to the path
with open(speech_file_path, 'wb') as file:
    file.write(response.content)

# Play the audio file
Audio(speech_file_path, autoplay=True)
